In [1]:
import sys
sys.path.insert(0, '..')
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json
import sys
import pandas as pd
import re

from config import reddit

In [2]:
from dataclasses import dataclass

@dataclass
class YoutubeLink:
    url: str
    subreddit: str
    thread_url: str
    
@dataclass
class TextEntry:
    text: str
    subreddit: str
    thread_url: str
    text_type: str

In [3]:
SUBREDDITS = reddit.SUBREDDITS
YoutubeLinks = []
TextEntries = []

driver = webdriver.Firefox()

In [9]:
def process_thread(url, subreddit):
    # old.reddit.com is easier to scrape, doesn't have js lazy loading
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    
    main_thread = soup.find('div', {'class': 'expando'}).text
    TextEntries.append(TextEntry(main_thread, subreddit, url, 'thread'))
    
    #Find all links in comments, keeping only youtube links
    comments = soup.find_all('div', {"data-type": "comment"})
    for comment in comments:
        #Record comment text
        comment_text = comment.find('div', {'class': 'md'}).text
        TextEntries.append(TextEntry(comment_text, subreddit, url, 'comment'))
        
        #Record youtube links
        for link in comment.find('div', {'class': 'usertext-body'}).find_all('a', href=True):
            if 'youtube.com' in link['href']:
                YoutubeLinks.append(YoutubeLink(link['href'], subreddit, url))
                print(link)


In [5]:
for SUBREDDIT in SUBREDDITS[:1]:
    url = f"https://reddit.com/r/{SUBREDDIT}/top/?sort=top&t=year"
    driver.get(url)
    
    # Get all thread links
    soup = BeautifulSoup(driver.page_source)
    links = soup.find_all('a', {"data-click-id": "body"})
    threads = [link for link in links if link['href'].startswith('/r/')]
    
    for thread in threads[:5]:
        thread_url = f"https://old.reddit.com{thread['href']}"
        links = process_thread(thread_url, SUBREDDIT)
                
driver.close()

KeyboardInterrupt: 

In [13]:
pd.DataFrame(YoutubeLinks).to_csv('../data/raw/reddit/scraped_youtube_links.csv', index=False)
pd.DataFrame(TextEntries).to_csv('../data/raw/reddit/scraped_comments.csv', index=False)